In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch
import pandas as pd
import numpy as np
import numpy as np
from os.path import join, basename, dirname, exists
from glob import glob
from ultralytics.utils.metrics import ConfusionMatrix
from ultralytics.utils.ops import xywhr2xyxyxyxy, xyxyxyxy2xywhr
from ultralytics import YOLO

## YOLO in-house models

In [ ]:
map50_fcbk = []
map50_zigzag = []
map75_fcbk = []
map75_zigzag = []
map5095_fcbk = []
map5095_zigzag = []
for fold in range(4):
    epochs = 100
    data_name = "lucknow" # "lucknow" or "wb_grid_400" or "delhi"
    task = "obb" # "obb" or aa"
    suffix = "v3" # "v1" or "v2" or "v3"
    yolo_task = "obb" # "obb" or "detect"
    model = "yolov8m-obb"
    gt_base_dir = "/path/data_download_experiments/sentinel_data/crossval"
    pred_base_dir = "/path/runs"
    data_folder = f"{data_name}_{task}_{suffix}"
    experiment_name = f"{data_folder}_model_{model}_epochs_{epochs}_sentinel_{fold}_predicted"

    model_path = join(pred_base_dir, yolo_task, experiment_name.replace("_predicted", ""), "weights", "best.pt")
    # model_path = ""
    model = YOLO(model_path, task="obb")
    metrics = model.val(data=join(gt_base_dir, data_folder, str(fold), "data.yml"), imgsz=1120, save_json=True)
    
    # get mAP50
    map50_fcbk.append(metrics.box.all_ap[0, 0])
    map50_zigzag.append(metrics.box.all_ap[1, 0])
    map75_fcbk.append(metrics.box.all_ap[0, 5])
    map75_zigzag.append(metrics.box.all_ap[1, 5])
    
    # get mAP50:95
    map5095_fcbk.append(metrics.box.all_ap.mean(axis=1)[0])
    map5095_zigzag.append(metrics.box.all_ap.mean(axis=1)[1])
    
    print("Fold", fold)
    print("mAP50 FCBK:", map50_fcbk[-1])
    print("mAP50 Zigzag:", map50_zigzag[-1])
    print("mAP75 FCBK:", map75_fcbk[-1])
    print("mAP75 Zigzag:", map75_zigzag[-1])
    print("mAP50:95 FCBK:", map5095_fcbk[-1])
    print("mAP50:95 Zigzag:", map5095_zigzag[-1])

In [ ]:
assert len(map50_fcbk) == 3
avg_map50_fcbk = np.mean(map50_fcbk)
avg_map50_zigzag = np.mean(map50_zigzag)
avg_map75_fcbk = np.mean(map75_fcbk)
avg_map75_zigzag = np.mean(map75_zigzag)
avg_map5095_fcbk = np.mean(map5095_fcbk)
avg_map5095_zigzag = np.mean(map5095_zigzag)

lst = [avg_map50_fcbk, avg_map50_zigzag, avg_map75_fcbk, avg_map75_zigzag, avg_map5095_fcbk, avg_map5095_zigzag]
print(" & ".join(map(lambda x: f"{x:.2f}", lst)))

In [ ]:
assert len(map50_fcbk) == 3
std_map50_fcbk = np.std(map50_fcbk)
std_map50_zigzag = np.std(map50_zigzag)
std_map75_fcbk = np.std(map75_fcbk)
std_map75_zigzag = np.std(map75_zigzag)
std_map5095_fcbk = np.std(map5095_fcbk)
std_map5095_zigzag = np.std(map5095_zigzag)

lst = [std_map50_fcbk, std_map50_zigzag, std_map75_fcbk, std_map75_zigzag, std_map5095_fcbk, std_map5095_zigzag]
print(" & ".join(map(lambda x: f"{x:.2f}", lst)))

In [ ]:
means = [avg_map50_fcbk, avg_map50_zigzag, avg_map5095_fcbk, avg_map5095_zigzag]
stds = [std_map50_fcbk, std_map50_zigzag,  std_map5095_fcbk, std_map5095_zigzag]
result = " & ".join([f"{mean:.2f} $\pm$ {std:.2f}" for mean, std in zip(means, stds)])
print(result)

## Outside the region evaluation

In [ ]:
map50_fcbk = []
map50_zigzag = []
map75_fcbk = []
map75_zigzag = []
map5095_fcbk = []
map5095_zigzag = []

epochs = 100
data_name = "lucknow"
task = "obb"
suffix = "v3"
yolo_task = "obb"
model = "yolov8l-obb"
gt_base_dir = "/path/kilns_neurips24/processed_data"
pred_base_dir = "/path/kilns_neurips24/runs/"
train_data_folder = f"lucknow_{task}_v3"
test_data_folder = f"wb_grid_400_{task}_v1"
train_experiment_name = f"{train_data_folder}_model_{model}_epochs_{epochs}"

model = YOLO(join(pred_base_dir, yolo_task, train_experiment_name, "weights", "best.pt"), task=yolo_task)
metrics = model.val(data=join(gt_base_dir, test_data_folder, "data.yml"), imgsz=1120, save_json=True)

# get mAP50
map50_fcbk.append(metrics.box.all_ap[0, 0])
map50_zigzag.append(metrics.box.all_ap[1, 0])
map75_fcbk.append(metrics.box.all_ap[0, 5])
map75_zigzag.append(metrics.box.all_ap[1, 5])

# get mAP50:95
map5095_fcbk.append(metrics.box.all_ap.mean(axis=1)[0])
map5095_zigzag.append(metrics.box.all_ap.mean(axis=1)[1])

print("mAP50 FCBK:", map50_fcbk[-1])
print("mAP50 Zigzag:", map50_zigzag[-1])
print("mAP75 FCBK:", map75_fcbk[-1])
print("mAP75 Zigzag:", map75_zigzag[-1])
print("mAP50:95 FCBK:", map5095_fcbk[-1])
print("mAP50:95 Zigzag:", map5095_zigzag[-1])

In [ ]:
avg_map50_fcbk = np.mean(map50_fcbk)
avg_map50_zigzag = np.mean(map50_zigzag)
avg_map75_fcbk = np.mean(map75_fcbk)
avg_map75_zigzag = np.mean(map75_zigzag)
avg_map5095_fcbk = np.mean(map5095_fcbk)
avg_map5095_zigzag = np.mean(map5095_zigzag)

lst = [avg_map50_fcbk, avg_map50_zigzag, avg_map75_fcbk, avg_map75_zigzag, avg_map5095_fcbk, avg_map5095_zigzag]
print(" & ".join(map(lambda x: f"{x:.2f}", lst)))